In [2]:
import pandas as pd
import numpy as np
from gmf import GMF
from mlp import MLP
from neumf import NeuMF
import torch.optim as optim
import math

In [3]:
ml1m_dir = 'data/yelp/ratings.txt'
ml1m_rating = pd.read_csv(ml1m_dir, sep='\t', header=None, names=['uid', 'rid', 'rating'],  engine='python')

In [4]:
user_id = ml1m_rating[['uid']].drop_duplicates()
item_id = ml1m_rating[['rid']].drop_duplicates()

In [5]:
user_id['uid'].max(), user_id['uid'].min(), user_id['uid'].shape

(122823, 0, (122677,))

In [6]:
item_id['rid'].max(), item_id['rid'].min(), item_id['rid'].shape

(28006, 0, (28002,))

In [7]:
num_users = user_id['uid'].max() - user_id['uid'].min() + 1
num_items = item_id['rid'].max() - item_id['rid'].min() + 1

In [8]:
subset = ml1m_rating[['uid', 'rid', 'rating']]
user_item_pairs = np.array([list(x)[0:2] for x in subset.values])
ratings = np.array([list(x)[2:3] for x in subset.values])

In [9]:
import torch
from torch.utils import data

class Dataset(data.Dataset):
  'Characterizes a dataset for PyTorch'
  def __init__(self, user_item_pairs, ratings):
        'Initialization'
        self.labels  = ratings
        self.samples = user_item_pairs

  def __len__(self):
        'Denotes the total number of samples'
        return len(self.samples)

  def __getitem__(self, index):
        'Generates one sample of data'
       # # Load data and get label
        #print("called get item")
        X = self.samples[index].astype('long')
        y = self.labels[index]
        return X, y

In [10]:
gmf_config = {'alias': 'train_gmf',
              'num_epoch': 8,
              'batch_size': 1024,
              'adam_lr': 1e-3,
              'num_users': num_users,
              'num_items': num_items,
              'latent_dim': 8,
              'l2_regularization': 0.01,
              'use_cuda': True,
              'device_id': 0}

In [11]:
mlp_config = {'alias': 'mlp_factor8neg4_bz256_166432168_pretrain_reg_0.0000001',
              'num_epoch': 5,
              'batch_size': 1024, 
              'num_users': num_users,
              'num_items': num_items,
              'latent_dim': 8,
              'layers': [16,32,16,8]}

In [13]:
neumf_config = {'alias': 'pretrain_neumf_factor8neg4',
                'num_epoch': 6,
                'batch_size': 1024,
                'num_users': num_users,
                'num_items': num_items,
                'latent_dim_mf': 8,
                'latent_dim_mlp': 8,
                'layers': [16,32,16,8],  # layers[0] is the concat of latent user vector & latent item vector
                'pretrain': False,
                'pretrain_mf': None,
                'pretrain_mlp': None
                }

In [14]:
train_ratings = list(np.load("train_ratings.npy"))
val_ratings = list(np.load("val_ratings.npy"))
test_ratings = list(np.load("test_ratings.npy"))

In [15]:
train_user_item_pairs = np.array([(x[0:2]) for x in train_ratings])
val_user_item_pairs = np.array([(x[0:2]) for x in val_ratings])
test_user_item_pairs = np.array([(x[0:2]) for x in val_ratings])
train_labels = np.array([(x[2:3]) for x in train_ratings])
val_labels = np.array([(x[2:3]) for x in val_ratings])
test_labels = np.array([(x[2:3]) for x in val_ratings])

In [16]:
training_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
training_set = Dataset(train_user_item_pairs, train_labels)
train_generator = data.DataLoader(training_set, **training_params)
val_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
val_set = Dataset(val_user_item_pairs, val_labels)
val_generator = data.DataLoader(val_set, **val_params)
test_params = {'batch_size': 1024,'shuffle': True,'num_workers': 0}
test_set = Dataset(test_user_item_pairs, test_labels)
test_generator = data.DataLoader(test_set, **test_params)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
def epoch_run(model, generator, opt, criterion,mode="train"):
    running_loss = 0
    if(mode == "train"):
        model.train()
    else:
        model.eval()
    for local_batch, local_labels in generator:
        local_batch  = torch.tensor(local_batch).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        y_preds = model(local_batch[:,0], local_batch[:,1])
        loss = criterion(y_preds, local_labels)
        running_loss += (loss.item()*local_labels.size()[0])
        if(mode == "train"):
            opt.zero_grad()
            loss.backward()
            opt.step()
    avg_loss = running_loss * 1.0 / (len(generator.dataset))
    return avg_loss

In [19]:
def predict(model, generator):
    model.eval()
    y_preds_all = torch.Tensor().to(device) 
    y_labels_all = torch.Tensor().to(device) 
    for local_batch, local_labels in generator:
        local_batch  = torch.tensor(local_batch).type(torch.long).to(device)
        local_labels = local_labels.type(torch.float).to(device)
        with torch.no_grad():
            y_preds = model(local_batch[:,0], local_batch[:,1])
        y_preds_all = torch.cat((y_preds_all,y_preds))
        y_labels_all = torch.cat((y_labels_all,local_labels))
    return y_preds_all, y_labels_all

In [20]:
import math
def evaluate(model, generator):
    y_preds_all, y_labels_all = predict(model, generator)  
    y_preds = list(y_preds_all.view(1, y_preds_all.size()[0]).to("cpu").numpy()[0])
    y_actuals = list(y_labels_all.view(1, y_labels_all.size()[0]).to("cpu").numpy()[0])
    #print(type(y_preds), type(y_actuals))
    tmse = sum([(a-b) * (a-b) for a,b in zip(y_preds, y_actuals)])
    rmse = math.sqrt((1.0*tmse)/len(y_preds))
    return rmse

In [21]:
def train_gmf():
    model = GMF(gmf_config).to(device)
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(gmf_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt, criterion, "train")
        val_mse = epoch_run(model, val_generator, opt, criterion,"val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [22]:
def train_mlp():
    model = MLP(mlp_config).to(device)
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(mlp_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt, criterion,"train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [23]:
def train_neumf():
    model = NeuMF(neumf_config).to(device)
#     if config['pretrain']:  #TODO:: Manoj
#         model.load_pretrain_weights()
    opt = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999),weight_decay=1e-4)
    criterion = torch.nn.MSELoss()
    for epoch in range(neumf_config['num_epoch']):
        print("running epoch ", epoch)
        train_mse = epoch_run(model, train_generator, opt,criterion, "train")
        val_mse = epoch_run(model, val_generator, opt,criterion, "val")
        print("train mse loss => ", train_mse, "val mse loss => ", val_mse)
    return model

In [22]:
gmf_model = train_gmf()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train mse loss =>  9.595478486335345 val mse loss =>  6.078701062905395
running epoch  1
train mse loss =>  4.1054407974208225 val mse loss =>  2.6434405063625026
running epoch  2
train mse loss =>  2.0464591744939744 val mse loss =>  1.6799269031932915
running epoch  3
train mse loss =>  1.6296891914957456 val mse loss =>  1.5921714111437466
running epoch  4
train mse loss =>  1.607191766190543 val mse loss =>  1.5916036133728044
running epoch  5
train mse loss =>  1.607115447435845 val mse loss =>  1.5916110017050518
running epoch  6
train mse loss =>  1.607120271086772 val mse loss =>  1.5916304270049932
running epoch  7
train mse loss =>  1.6071295327888098 val mse loss =>  1.5916016936755122


In [23]:
mlp_model = train_mlp()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train mse loss =>  2.088761404724617 val mse loss =>  1.5956651328082017
running epoch  1
train mse loss =>  1.5162342781998275 val mse loss =>  1.4650156432668133
running epoch  2
train mse loss =>  1.2962417460996116 val mse loss =>  1.3326339979284552
running epoch  3
train mse loss =>  1.1947617218902533 val mse loss =>  1.3236104126429726
running epoch  4
train mse loss =>  1.1414955583747792 val mse loss =>  1.3322711227758965


In [24]:
neumf_model = train_neumf()

running epoch  0


/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


train mse loss =>  2.2441500270257877 val mse loss =>  1.5899769734830684
running epoch  1
train mse loss =>  1.5017119352592165 val mse loss =>  1.4346645324747147
running epoch  2
train mse loss =>  1.2849891046653534 val mse loss =>  1.326437991741831
running epoch  3
train mse loss =>  1.193985375598415 val mse loss =>  1.3179139418759134
running epoch  4
train mse loss =>  1.1432111996668568 val mse loss =>  1.322859048140291
running epoch  5
train mse loss =>  1.0794715754896362 val mse loss =>  1.3427946512008124


In [30]:
torch.save(gmf_model.state_dict(), "./saved_models/gmf.dict")

In [31]:
torch.save(mlp_model.state_dict(), "./saved_models/mlp.dict")

In [32]:
torch.save(neumf_model.state_dict(), "./saved_models/neumf.dict")

In [35]:
model = NeuMF(neumf_config).to(device)
model.load_state_dict(torch.load("./saved_models/neumf.dict"))
model.eval()

NeuMF(
  (embedding_user_mlp): Embedding(122824, 8)
  (embedding_item_mlp): Embedding(28007, 8)
  (embedding_user_mf): Embedding(122824, 8)
  (embedding_item_mf): Embedding(28007, 8)
  (fc_layers): ModuleList(
    (0): Linear(in_features=16, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=8, bias=True)
  )
  (affine_output): Linear(in_features=16, out_features=1, bias=True)
)

In [50]:
evaluate(model, test_generator)

/home/forcerequestspring19_gmail_com/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


1.1632848231420039